In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb

In [ ]:
dataset = pd.read_csv('train.csv')

In [117]:
FEATURE_SIZE=3
X = dataset.iloc[:,0:FEATURE_SIZE].values
print(len(X))
y = dataset.iloc[:, 7].values

print(len(y))

99968
99968


In [118]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.1)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size = 0.1)


In [119]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_val=sc.transform(x_val)

In [120]:
train_data = lgbm.Dataset(x_train, y_train)
test_data = lgbm.Dataset(x_test, y_test)

parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'false',
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 5,
    'num_threads' : 2,
    'seed' : 76,
    "max_bin": 100,
    "num_iterations": 100,
    "max_depth": 5
}


lr = lgb.train(params, train_data, 20,test_data)

[1]	valid_0's auc: 0.861563
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.861563
[3]	valid_0's auc: 0.861595
[4]	valid_0's auc: 0.861767
[5]	valid_0's auc: 0.861767
[6]	valid_0's auc: 0.861767
[7]	valid_0's auc: 0.861776
[8]	valid_0's auc: 0.861779
[9]	valid_0's auc: 0.861779
[10]	valid_0's auc: 0.861794
[11]	valid_0's auc: 0.861791
[12]	valid_0's auc: 0.861791
[13]	valid_0's auc: 0.861794
[14]	valid_0's auc: 0.861808
[15]	valid_0's auc: 0.861808
[16]	valid_0's auc: 0.861817
[17]	valid_0's auc: 0.8618
[18]	valid_0's auc: 0.8618
[19]	valid_0's auc: 0.8618
[20]	valid_0's auc: 0.861794
Did not meet early stopping. Best iteration is:
[16]	valid_0's auc: 0.861817


H:\Anaconda\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [121]:
y_pred=lr.predict(test_data)
for i in range(len(y_pred)):
    if y_pred[i]>=.5:       
        y_pred[i]=1
    else:
        y_pred[i]=0
    
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)
print(accuracy)

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred))

0.833
0.833


In [123]:
testdata = pd.read_csv('test.csv')
x_testing = testdata.iloc[:,0:FEATURE_SIZE].values
x_testing = sc.transform(x_testing)
y_pred=lr.predict(x_testing)
predictions_link = np.array(y_pred)
print(len(predictions_link))
idx = np.argpartition(predictions_link, 1000)
print(max(predictions_link[idx[:1000]]))

1999
0.6356612143678946


In [124]:
import csv
count1 = 0
count0 = 0

with open("Light:(0-1.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Predicted"])
    test_id=1
    for prediction in y_pred:
        if prediction > 0.8:
            writer.writerow([test_id,1])
            count1 += 1
        elif prediction < 0.2:
            writer.writerow([test_id,0])
            count0 += 1
        else:
            writer.writerow([test_id,prediction])
        test_id+=1

In [125]:
print(count1)
print(count0)

0
0
